```
This notebook sets up and runs a test case for analyzing Kelvin waves
Copyright (C) 2018 - 2022 SINTEF Digital
Copyright (C) 2018 - 2022 Norwegian Meteorological Institute

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

In [1]:
import sys
gpuocean_path = [p[:-4] for p in sys.path if p.endswith("gpuocean/src")][0]
import git
repo = git.Repo(gpuocean_path)
print("GPUOcean code from:", repo.head.object.hexsha, "on branch", repo.active_branch.name)

GPUOcean code from: 0bff8670195d21d3f7f0df772874a6ca770b6fe0 on branch spatial_gravity


Showcasing the improved handling of potential energy fluxes along the land mask!

In [2]:
#Lets have matplotlib "inline"
%matplotlib inline

import os
import sys

#Import packages we need
import numpy as np
from netCDF4 import Dataset
import datetime, copy
from IPython.display import display

#For plotting
import matplotlib
from matplotlib import pyplot as plt

plt.rcParams["lines.color"] = "w"
plt.rcParams["text.color"] = "w"
plt.rcParams["axes.labelcolor"] = "w"
plt.rcParams["xtick.color"] = "w"
plt.rcParams["ytick.color"] = "w"

plt.rcParams["image.origin"] = "lower"

In [3]:
from gpuocean.utils import IPythonMagic

In [4]:
%cuda_context_handler gpu_ctx

#### Simple example with constant non-zero bathymetry

Set up

In [5]:
nx=100
ny=100

dx=100
dy=100

dt=0.0

g=1.0
f=0.0
r=0.0

In [6]:
eta0 = 3 * np.ones((ny+4,nx+4))
hu0 = np.ones((ny+4,nx+4))
hv0 = np.zeros((ny+4,nx+4))

In [7]:
Hm_data = 2*np.ones((ny+6,nx+6))
Hm_mask = np.ones((ny+6,nx+6), dtype=bool)
Hm_mask[28:78,28:78] = False
Hm = np.ma.array(Hm_data, mask=Hm_mask)

from gpuocean.utils import OceanographicUtilities

Hi = OceanographicUtilities.midpointsToIntersections(Hm, land_value=0.0, iterations=10)[0].astype(np.float32)


Running the simulation

In [8]:
from gpuocean.SWEsimulators import CDKLM16
from importlib import reload
reload(CDKLM16)

<module 'gpuocean.SWEsimulators.CDKLM16' from '/home/florianb/havvarsel/gpuocean/src/gpuocean/SWEsimulators/CDKLM16.py'>

In [9]:
from gpuocean.SWEsimulators import CDKLM16
sim = CDKLM16.CDKLM16(gpu_ctx, eta0, hu0, hv0, Hi, nx, ny, dx, dy, dt, g, f, r, rk_order=1)

In [10]:
sim.gpu_data.h0.data.get()[24:29,24:29], sim.gpu_data.hu0.data.get()[24:29,24:29]

(array([[3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3.]], dtype=float32),
 array([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]], dtype=float32))

In [11]:
sim.step(0.1)

0.10000000149011612

The first 2 rows and columns are DRY.

In [12]:
sim.gpu_data.h0.data.get()[24:29,24:29]

array([[1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05],
       [1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05],
       [1.0000e-05, 1.0000e-05, 2.9992e+00, 3.0000e+00, 3.0000e+00],
       [1.0000e-05, 1.0000e-05, 2.9990e+00, 3.0000e+00, 3.0000e+00],
       [1.0000e-05, 1.0000e-05, 2.9990e+00, 3.0000e+00, 3.0000e+00]],
      dtype=float32)

In [13]:
sim.gpu_data.hu0.data.get()[24:29,24:29]

array([[0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.69984   , 0.8       , 0.8       ],
       [0.        , 0.        , 0.79980004, 1.        , 1.        ],
       [0.        , 0.        , 0.79980004, 1.        , 1.        ]],
      dtype=float32)

In [14]:
sim.gpu_data.hv0.data.get()[24:29,24:29]

array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, -7.4505807e-10,
        -7.4505807e-10],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00]], dtype=float32)

The 2 last rows and columns are DRY 

In [15]:
sim.gpu_data.h0.data.get()[75:80,75:80]

array([[3.0000e+00, 3.0000e+00, 3.0010e+00, 1.0000e-05, 1.0000e-05],
       [3.0000e+00, 3.0000e+00, 3.0010e+00, 1.0000e-05, 1.0000e-05],
       [3.0000e+00, 3.0000e+00, 3.0008e+00, 1.0000e-05, 1.0000e-05],
       [1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05],
       [1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05]],
      dtype=float32)

In [16]:
sim.gpu_data.hu0.data.get()[75:80,75:80]

array([[1.        , 1.        , 0.8002    , 0.        , 0.        ],
       [1.        , 1.        , 0.8002    , 0.        , 0.        ],
       [0.8       , 0.8       , 0.70015997, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ]],
      dtype=float32)

In [17]:
sim.gpu_data.hv0.data.get()[75:80,75:80]

array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00],
       [7.4505807e-10, 7.4505807e-10, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00]], dtype=float32)

In that experiment, we get beside numerical effects also some mathematical effect from the bathymetry forcing along the land mask cells!
In the next case, we reduce the bathymetry effects even further! 

#### Same example with 0-bathymetry

In [18]:
eta0 = 5 * np.ones((ny+4,nx+4))

In [19]:
Hm_data = np.zeros((ny+6,nx+6))
Hm_mask = np.ones((ny+6,nx+6), dtype=bool)
Hm_mask[28:78,28:78] = False
Hm = np.ma.array(Hm_data, mask=Hm_mask)

from gpuocean.utils import OceanographicUtilities

Hi = OceanographicUtilities.midpointsToIntersections(Hm, land_value=0.0, iterations=10)[0].astype(np.float32)

In [20]:
from gpuocean.SWEsimulators import CDKLM16
from importlib import reload
reload(CDKLM16)

<module 'gpuocean.SWEsimulators.CDKLM16' from '/home/florianb/havvarsel/gpuocean/src/gpuocean/SWEsimulators/CDKLM16.py'>

In [21]:
from gpuocean.SWEsimulators import CDKLM16
sim = CDKLM16.CDKLM16(gpu_ctx, eta0, hu0, hv0, Hi, nx, ny, dx, dy, dt, g, f, r, rk_order=1)

In [22]:
sim.step(0.1)

0.10000000149011612

The first 2 rows and columns are DRY

In [23]:
sim.gpu_data.h0.data.get()[24:29,24:29]

array([[1.000e-05, 1.000e-05, 1.000e-05, 1.000e-05, 1.000e-05],
       [1.000e-05, 1.000e-05, 1.000e-05, 1.000e-05, 1.000e-05],
       [1.000e-05, 1.000e-05, 4.999e+00, 5.000e+00, 5.000e+00],
       [1.000e-05, 1.000e-05, 4.999e+00, 5.000e+00, 5.000e+00],
       [1.000e-05, 1.000e-05, 4.999e+00, 5.000e+00, 5.000e+00]],
      dtype=float32)

In [24]:
sim.gpu_data.hu0.data.get()[24:29,24:29]

array([[0.    , 0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.9998, 1.    , 1.    ],
       [0.    , 0.    , 0.9998, 1.    , 1.    ],
       [0.    , 0.    , 0.9998, 1.    , 1.    ]], dtype=float32)

In [25]:
sim.gpu_data.hv0.data.get()[24:29,24:29]

array([[ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00,
         0.000000e+00],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00,
         0.000000e+00],
       [ 0.000000e+00,  0.000000e+00, -9.536744e-10, -9.536744e-10,
        -9.536744e-10],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00,
         0.000000e+00],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00,
         0.000000e+00]], dtype=float32)

The 2 last rows and columns are DRY 

In [26]:
sim.gpu_data.h0.data.get()[75:80,75:80]

array([[5.000e+00, 5.000e+00, 5.001e+00, 1.000e-05, 1.000e-05],
       [5.000e+00, 5.000e+00, 5.001e+00, 1.000e-05, 1.000e-05],
       [5.000e+00, 5.000e+00, 5.001e+00, 1.000e-05, 1.000e-05],
       [1.000e-05, 1.000e-05, 1.000e-05, 1.000e-05, 1.000e-05],
       [1.000e-05, 1.000e-05, 1.000e-05, 1.000e-05, 1.000e-05]],
      dtype=float32)

In [27]:
sim.gpu_data.hu0.data.get()[75:80,75:80]

array([[1.    , 1.    , 1.0002, 0.    , 0.    ],
       [1.    , 1.    , 1.0002, 0.    , 0.    ],
       [1.    , 1.    , 1.0002, 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    ]], dtype=float32)

In [28]:
sim.gpu_data.hv0.data.get()[75:80,75:80]

array([[0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00],
       [9.536744e-10, 9.536744e-10, 9.536744e-10, 0.000000e+00,
        0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00]], dtype=float32)